For each intermediate stop on a tour (i.e. trip other than the last trip outbound or inbound) each trip is assigned a purpose based on an observed frequency distribution.
The distribution is segmented by tour purpose, tour direction and person type. Work tours are also segmented by departure or arrival time period.

In [1]:
import os
import toml
import pandas as pd
import numpy as np
import plotly.express as px
import validation_data_input
import psrc_theme


import plotly.io as pio
pio.renderers.default = "plotly_mimetype+notebook_connected" # to show plotly figures in quarto HTML file
pio.templates.default = "simple_white+psrc_color" # set plotly template

config = toml.load(os.path.join(os.getcwd(), 'validation_configuration.toml'))

In [2]:
# read data
per_data = validation_data_input.get_persons_data(col_list=['ptype'])
tour_data = validation_data_input.get_tours_data(col_list=['tour_type','start','end'])
trip_data = validation_data_input.get_trips_data()

# add person type labels
ptype_cat = {1: "1: Full-Time Worker",
             2: "2: Part-Time Worker",
             3: "3: University Student",
             4: "4: Non-Working Adult Age <65",
             5: "5: Non-Working Adult Age 65+",
             6: "6: High School Student Age 16+",
             7: "7: Child Age 5-15",
             8: "8: Child Age 0-4"}
per_data['ptype_label'] = per_data['ptype'].map(ptype_cat)
trip_data['purpose'] = trip_data['purpose'].apply(lambda x: "home" if x == "Home" else x)
trip_data['tour_direction'] = trip_data['outbound'].apply(lambda x: "outbound" if x == True else "inbound")

tour_data = tour_data.merge(per_data, how='left', on=['person_id','household_id','source'])

trip_data = trip_data.merge(tour_data, how='left',on=['person_id','household_id','tour_id','trip_weight_2017_2019','source'])

In [4]:
df_plot = trip_data.groupby(['source','purpose'])['trip_weight_2017_2019'].sum().reset_index()
df_plot['percentage'] = df_plot.groupby(['source'], group_keys=False)['trip_weight_2017_2019']. \
    apply(lambda x: x / float(x.sum()))
# df_plot
fig = px.bar(df_plot, x="purpose", y="percentage", color="source",barmode="group",
             title="Trip purpose")
fig.for_each_annotation(lambda a: a.update(text = a.text.split("=")[-1]))
fig.update_layout(height=400, width=700, yaxis=dict(tickformat=".1%"))
fig.show()

## Trip purpose by segment

In [5]:
def plot_purpose(df: pd.DataFrame, grp_var: str, grp_var_name: str, n_nol: int, height: int):
    df_plot = df.groupby(['source',grp_var,'purpose'])['trip_weight_2017_2019'].sum().reset_index()
    df_plot['percentage'] = df_plot.groupby(['source',grp_var], group_keys=False)['trip_weight_2017_2019']. \
        apply(lambda x: x / float(x.sum()))

    fig = px.bar(df_plot,
                 x="percentage", y="purpose", color="source",barmode="group",
                 facet_col=grp_var, facet_col_wrap=n_nol, orientation='h',
                 title="Trip purpose by " + grp_var_name)
    fig.for_each_annotation(lambda a: a.update(text = a.text.split("=")[-1]))
    fig.update_layout(height=height, width=770)
    fig.for_each_xaxis(lambda a: a.update(tickformat = ".1%"))
    fig.update_yaxes()
    fig.show()

In [6]:
plot_purpose(trip_data,'ptype_label','person type',3,1200)

In [7]:
plot_purpose(trip_data,'tour_type','tour purpose',3,1200)

In [8]:
plot_purpose(trip_data,'tour_direction','tour direction',2,500)